# 1. Introduction (**DOES NOT WORK, INTRODUCE MF-GPy FIRST**)

This notebook is dedicated to outline the issues surrounding the hyperparameter optimization (HPO) of Gaussian processes in the context of Multi-fidelity Gaussian process regression (MFGPR); in particular the estimation of Gaussian noise variance based on the provided DoE.

It is essential to have a properly functioning MFGPR-HPO framework if Multi-fidelity Bayesian optimization (MFBO) is to be developed.

Throughout the notebook, [Taylan's GPy-MFGPR implementation](https://github.com/taylanot/GPy) (latest version if possible) shall be considered.

The following packages are needed:

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import GPy.models

np.random.seed(123) # RNG control parameter

Next, we will define some test functions that have been used in _(cite papers)_.

In [7]:
################################################################################
# Expensive Function
def fe(x):
    return (6.0 * x - 2.) ** 2 * np.sin(12 * x - 4) + 0.1

# Cheap Function
def fc(x):
    A = 0.5
    B = 10
    C = 5
    return A * fe(x) + B * (x - 0.5) - C + 0.2
################################################################################

x = np.linspace(0, 1, 100).reshape(-1, 1)

We will now introduce some control parameters that will act to show the "undesirable" behavior of the MFGPR-HPO.

In [8]:
optimizer_bool = True
optimizer_string = 'lbfgsb'
num_of_restarts = 10
DoE_set = 1

noise_var_lf = 0.5
noise_var_hf = 0

The training data (input and output) the MFGPR experiment are as follows.

In [9]:
if DoE_set == 1:
    Xl = np.linspace(0, 1, 11).reshape(-1, 1)
    Xh = np.array([0, 0.4, 0.6, 0.8, 1]).reshape(-1, 1)

if DoE_set == 2:
    Xl = np.linspace(0, 1, 6).reshape(-1, 1)
    Xh = np.array([0, 0.4, 1]).reshape(-1, 1)

X = [Xl, Xh]

Yl = fc(Xl)
Yh = fe(Xh)

Y = [Yl, Yh]

Next, the multi-fidelity Gaussian process surface modeling takes place.

In [10]:
m = GPy.models.multiGPRegression(X, Y)

AttributeError: module 'GPy.models' has no attribute 'multiGPRegression'

Choice of HP optimizer. 

In [ ]:
m.models[0].preferred_optimizer = optimizer_string
m.models[1].preferred_optimizer = optimizer_string

Choice whether constraints will be applied to the HPs.

In [ ]:
# m.models[0]['Gaussian_noise.variance'].constrain_bounded(0, 1)
# m.models[0]['rbf.variance'].constrain_bounded(1, 20)
# m.models[0]['rbf.lengthscale'].constrain_bounded(0.1, 5)

# m.models[1]['Gaussian_noise.variance'].constrain_bounded(0, 1)
# m.models[1]['rbf.variance'].constrain_bounded(1, 5)
# m.models[1]['rbf.lengthscale'].constrain_bounded(0.1, 5)

Choice of which HPs to fix. 

In [ ]:
m.models[0]['Gaussian_noise.variance'].fix(noise_var_lf)
# m.models[0]['rbf.variance'].fix(1.5)
# m.models[0]['rbf.lengthscale'].fix(0.1)

m.models[1]['Gaussian_noise.variance'].fix(noise_var_hf)
# m.models[1]['rbf.variance'].fix(0.1)
# m.models[1]['rbf.lengthscale'].fix(0.1)

Next is the HPO scheme. This will optimize the remaining, non-fixed HPs.

In [ ]:
if optimizer_bool:
    m.optimize_restarts(restarts=num_of_restarts, verbose=False)

print(m.models[1].log_likelihood())

print(m)

The resulting GPs are plotted against the true functions.

In [ ]:
### Prediction (MAKE SURE ALL HYPERPARAMETERS ARE SET CORRECTLY)
mu, sigma = m.predict(x)

### Visualization
vis = True
if vis:
    plt.plot(x, mu[0], color='b', label='MF cheap GPR (regular GPR)')
    plt.plot(x, mu[0] + 2 * sigma[0], color='k', lw=.5)
    plt.plot(x, mu[0] - 2 * sigma[0], color='k', lw=.5)
    plt.fill_between(x.flatten(), mu[0].flatten() - 2 * sigma[0].flatten(), mu[0].flatten() + 2 * sigma[0].flatten(), alpha=0.2, color='b')

    # plt.plot(x, mu_par, color='r', label='Regular GPR', alpha=0.3)
    # plt.plot(x, mu_par + 2 * sigma_par, color='k')
    # plt.plot(x, mu_par - 2 * sigma_par, color='k')
    # plt.fill_between(x.flatten(), mu_par.flatten() - 2 * sigma_par.flatten(), mu_par.flatten() + 2 * sigma_par.flatten(), alpha=0.2)

    plt.plot(x, mu[1], color='orange', label='MF expensive GPR')
    plt.plot(x, mu[1] + 2 * sigma[1], color='k', lw=.5)
    plt.plot(x, mu[1] - 2 * sigma[1], color='k', lw=.5)
    plt.fill_between(x.flatten(), mu[1].flatten() - 2 * sigma[1].flatten(), mu[1].flatten() + 2 * sigma[1].flatten(), alpha=0.2, color='orange')

    plt.plot(x, fc(x), '--', color='b', label='Exact cheap function')
    plt.plot(x, fe(x), '--', color='orange', label='Exact expensive function')

    plt.legend()

    plt.scatter(Xl, Yl, color='b')
    plt.scatter(Xh, Yh, color='orange')

    plt.grid()

    # plt.savefig('noise_experiment_Opt-%s_DoE%s_%s_%s.svg' % (optimizer_string, DoE_set, noise_var_lf, noise_var_hf))
    # m.plot()
    plt.show()